In [65]:
import pandas as pd
import sys, os
import numpy as np
from tqdm import tqdm

In [47]:
folder_path = "data/Daejeon/2022/"
folder_names = [folder for folder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, folder))]

In [44]:
#get node
nlist = []
for fname in folder_names:
    file_path = folder_path + fname
    file_names = os.listdir(file_path)
    file_path += '/' + file_names[0]
    df = pd.read_csv(file_path)
    df.columns = ['Time', 'LINK_ID', 'Speed']
    unique_link_ids = df['LINK_ID'].nunique()
    if unique_link_ids == 3476:
        col = df['LINK_ID'].drop_duplicates().values
        break

In [45]:
start = "2022-01-01 00:00:00"
end = "2022-06-30 23:55:00"
date_index = pd.date_range(start, end, freq='5T')
speed_csv = pd.DataFrame(columns = col, index=date_index)

In [61]:
for fname in folder_names:
    file_path = folder_path + fname
    file_names = os.listdir(file_path)
    file_path += '/' + file_names[0]
    df = pd.read_csv(file_path)
    df.columns = ['Time', 'LINK_ID', 'Speed']
    df['Time'] = pd.to_datetime(df['Time'], format='%Y%m%d%H%M')
    for t, lid, spd in zip(df['Time'], df['LINK_ID'], df['Speed']):
        speed_csv.loc[t, lid] = spd

In [67]:
rlist = []
for i in tqdm(range(3480)):
    cnt = 0
    for j in range(52128):
        if np.isnan(speed_csv.iloc[j, i]):
            cnt += 1
        if cnt == 1000:
            rlist.append(i)
            break

100%|██████████████████████████████████████████████████████████████████████████████| 3480/3480 [06:43<00:00,  8.62it/s]


In [70]:
len(rlist)

3057

In [73]:
speed_csv.to_csv('data/Daejeon/speed_all_node.csv')

In [74]:
speed_csv.drop(speed_csv.columns[rlist], axis=1, inplace=True)

In [76]:
speed_csv.to_csv('data/Daejeon/speed.csv')
speed_csv_imputed = speed_csv.copy()

In [ ]:
# Non observation count
cnt = 0
for i in tqdm(range(len(speed_csv.columns))):
    for j in range(len(speed_csv.index)):
        if np.isnan(speed_csv.iloc[j,i]):
            cnt += 1
print(cnt)

In [199]:
# Imputation with the average rate of change compared to the previous time slot for the same day of the week and the same time.
for i in tqdm(range(len(speed_csv.columns))):
    for j in range(len(speed_csv.index)):
        if np.isnan(speed_csv.iloc[j, i]):
            temp = j
            cnt = 0
            change_rate_sum = 0
            for k in range(25):
                temp += 2016
                if not np.isnan(speed_csv.iloc[temp % len(speed_csv.index), i]):
                    if not np.isnan(speed_csv.iloc[(temp % len(speed_csv.index)) - 1, i]):
                        cnt += 1
                        prev_speed = speed_csv.iloc[(temp % len(speed_csv.index)) - 1, i]
                        current_speed = speed_csv.iloc[temp % len(speed_csv.index), i]
                        if prev_speed != 0:
                            diff = prev_speed - current_speed
                            change_rate_sum += diff / prev_speed

            if cnt > 0:
                imputed_speed = speed_csv.iloc[j - 1, i] * (1 + change_rate_sum / cnt)
                speed_csv_imputed.iloc[j, i] = imputed_speed
                if imputed_speed < 0 or imputed_speed > 135:
                    print(speed_csv.iloc[j - 1, i], (1 + change_rate_sum / cnt))
                    print(change_rate_sum, cnt, j, i)

In [ ]:
# After the first imputation, count the missing values.
cnt = 0
for i in tqdm(range(len(speed_csv_imputed.columns))):
    for j in range(len(speed_csv_imputed.index)):
        if np.isnan(speed_csv_imputed.iloc[j,i]):
            cnt += 1
print(cnt)

In [205]:
# After the first imputation, handle the remaining unobserved values by imputing them with the median of the same day of the week and the same time slot.
for i in tqdm(range(len(speed_csv.columns))):
    for j in range(len(speed_csv.index)):
        if np.isnan(speed_csv.iloc[j, i]):
            temp = j
            num_list = []
            for k in range(25):
                temp += 2016
                if not np.isnan(speed_csv.iloc[temp % len(speed_csv.index), i]):
                    num_list.append(speed_csv.iloc[temp % len(speed_csv.index), i])
            median = sorted(num_list)[len(num_list)//2]
            
            speed_csv_imputed.iloc[j, i] = median

100%|████████████████████████████████████████████████████████████████████████████████| 423/423 [04:40<00:00,  1.51it/s]


In [207]:
speed_csv_imputed.to_csv('data/Daejeon/speed_imputed.csv')

In [209]:
speed_csv_imputed.index = pd.to_datetime(speed_csv_imputed.index)

In [211]:
speed_csv_imputed.to_hdf('data/Daejeon/Daejeon.h5', 'df', format='table')

In [215]:
speed_csv_imputed

,3391442,3402981,3404081,3404102,3434152,3434162,3439012,3440371,3442932,3444332,...,3358341,3388262,3402842,3440242,16004891,10194532,10424801,3428961,3386892,3388011
2022-01-01 00:00:00,58.0,53.0,55.0,79.0,65.0,85.0,81.0,48.0,86.0,93.0,...,38.0,23.0,54.0,56.0,64.0,89.0,58.0,62.0,12.0,28.0
2022-01-01 00:05:00,59.0,52.0,38.0,73.0,73.0,79.0,73.0,53.0,93.0,107.0,...,36.0,33.0,52.0,57.0,54.0,96.0,55.0,47.0,19.0,40.0
2022-01-01 00:10:00,60.0,57.0,51.0,75.0,71.0,80.0,58.0,55.0,92.0,113.0,...,33.0,38.0,40.0,59.0,54.0,93.0,55.0,42.0,53.0,42.0
2022-01-01 00:15:00,60.0,57.0,57.0,76.0,68.0,79.0,82.0,54.0,94.0,100.0,...,41.0,24.0,51.0,59.0,62.0,94.0,48.0,59.0,24.0,43.0
2022-01-01 00:20:00,57.0,54.0,55.0,74.0,68.0,87.0,82.0,52.0,96.0,108.0,...,15.0,25.0,16.0,54.0,50.0,92.0,52.0,68.0,16.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 23:35:00,58.0,55.0,39.0,74.0,61.0,72.0,76.0,47.0,92.0,98.0,...,34.0,29.0,53.0,53.0,63.0,89.0,47.0,55.0,17.0,11.0
2022-06-30 23:40:00,57.0,7.0,44.0,76.0,58.0,82.0,87.0,49.0,89.0,100.0,...,43.0,37.0,10.0,58.0,60.0,95.0,53.0,34.0,27.0,20.0
2022-06-30 23:45:00,59.0,24.0,41.0,74.0,60.0,85.0,89.0,50.0,86.0,105.0,...,39.0,28.0,35.0,58.0,60.0,94.0,51.0,54.0,27.0,34.0
2022-06-30 23:50:00,59.0,39.0,38.0,77.0,56.0,85.0,90.0,50.0,93.0,96.0,...,15.0,41.0,45.0,58.0,60.0,89.0,53.0,56.0,19.0,23.0
